In [1]:
from parser_1 import Parser
from typing import List


import pandas as pd

In [11]:

class Groupper(Parser):

    @staticmethod
    def _get_output_balance(df: pd.DataFrame) -> pd.DataFrame:
        df['output_active'] = df['active'] + df['debit'] - df['credit']
        df.loc[df['active']  == 0, 'output_active'] = 0

        df['output_passive'] = df['passive'] - df['debit'] + df['credit']
        df.loc[df['passive']  == 0, 'output_passive'] = 0
        return df.astype('float128')
    
    def add_group_result(self, df: pd.DataFrame) -> pd.DataFrame:
        conc_df = pd.DataFrame()
        for id, gr_df in df.groupby(df.unid.apply(lambda a: int(a) // 100)):
            res = gr_df.sum(axis=0)
            res.unid = id
            conc_df = pd.concat([conc_df, gr_df, pd.DataFrame(res).T]).reset_index(drop=True)
        return conc_df
    
    def add_class_result(self, df: pd.DataFrame) -> pd.DataFrame:
        result = pd.DataFrame(df[df.unid > 99].sum(axis=0)).T
        result.unid = "ИТОГО ЗА КЛАСС"
        return pd.concat([df, result]).reset_index(drop=True)
    
    def add_total_result(self, dfs: List[pd.DataFrame]) -> pd.DataFrame:
        df = pd.concat([self.add_class_result(
                            self.add_group_result(
                                self._get_output_balance(df_)
                                )
                            ) 
                        for df_ in dfs])
        result = pd.DataFrame(df[df.unid == 'ИТОГО ЗА КЛАСС'].sum(axis=0)).T
        result.unid = "ИТОГО"
        return pd.concat([df, result]).reset_index(drop=True)
    
    def get_grouped_data(self, path: str) -> pd.DataFrame:
        dfs = self.get_classes_from_excel_file(path)
        return self.add_total_result(dfs)

In [12]:
gpr = Groupper()

In [13]:
gpr.get_grouped_data('files/ОСВ.xlsx')

,unid,active,passive,debit,credit,output_active,output_passive
0,1019.0,74975624165.882996,0.0,15640681705689.0,14312543941884.0,1403113387970.882812,0.0
1,1011.0,58496794456.711998,0.0,689834244241.75,34144530376697.0,-33396199337998.539062,0.0
2,1020.0,16521001749.354,0.0,654759955289.73999,3865541431398.299805,-3194260474359.206055,0.0
3,1030.0,20431402379.548,0.0,9951931131495.599609,5748850039946.0,4223512493929.148438,0.0
4,1040.0,115251239713.380005,0.0,9668096244082.0,9866340607336.699219,-82993123541.318359,0.0
...,...,...,...,...,...,...,...
604,95.0,0.0,0.0,31974517.765367,39614405.882501,0.0,0.0
605,9600.0,0.0,0.0,6881896495.3624,31176455711.152,0.0,0.0
606,96.0,0.0,0.0,6881896495.3624,31176455711.152,0.0,0.0
607,ИТОГО ЗА КЛАСС,0.0,0.0,2591069446073.843262,4462788902933.989258,0.0,0.0
